# It could be cake <div style="text-align: right; float: right"> IANNWTF - Final Task </div>

##### The usual imports

In [1]:
import numpy as np
import os, time
import tensorflow as tf
import json

C:\Users\nipsh\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


##### Recipe data script

In [2]:
# %load recipes.py
import pymongo, json, numpy as np
from collections import Counter
from nltk.tokenize import RegexpTokenizer


class Recipes:

    def __init__(
      self,
      useMongo=True,
      config="config.json"
    ):

        with open(config) as cfg:
            self.data = json.load(cfg)
            storage = self.data['storage']
            CRAWL_FOLDER = storage['folder']
            RECIPE_COLLECTION = storage['collection']

        client = pymongo.MongoClient('mongodb://localhost:27017')
        self.db = client['iannwtf']
        self.coll = self.db[RECIPE_COLLECTION]

    def create_dictionaries(self,vocabulary_size=20000):
        agg = self.coll.aggregate([
            {"$project": {"_id": 0, "ingredients.foodId": 1, "ingredients.name": 1}},
            {"$unwind": "$ingredients"},
            {
                "$group": {
                    "_id": "$ingredients.foodId",
                    "name": {"$first": "$ingredients.name"}
                }
            }
          ], allowDiskUse=True)
    
    

        self._ing2id = {item['name'].lower(): int(item['_id']) for item in agg}
        self._ing2id['miracel whipmyass'] = 1362
        self._ing2id['NO_INGREDIENT'] = 0
        self._id2ing = dict(zip(self._ing2id.values(), self._ing2id.keys()))

        self._word2id = {}
        self._id2word = {} 
        self._unit2id = {}
        self._id2unit = {}

        ingredient_size = len(self._ing2id)
        unit_size = len(self._unit2id)

        return ingredient_size, unit_size, vocabulary_size

    def ings2ids(self, ings):
        return self._2(ings, self._ing2id, 0)
  
    def units2ids(self, units):
        return self._2(units, self._unit2id, 0)

    def words2ids(self, words):
        return self._2(words, self._word2id, 0)

    def ids2ings(self, ids):
        return self._2(ids, self._id2ing, "NO_INGREDIENT")
  
    def ids2units(self, ids):
        return self._2(ids, self._id2unit, "NO_UNIT")

    def ids2words(self, ids):
        return self._2(ids, self._id2word, "UNKNOWN")

    def _2(self, names, d, undef):
        if type(names) in [list, range]:
            return [d.get(n, undef) for n in names]
        elif type(names) == np.ndarray:
            res = np.full_like(names, "", dtype=object)
            for i in range(res.shape[0]):
                res[i] = self._2(names[i], d, undef)
            return res
        else:
            return d.get(names, undef)

    def get_ingredient_batch(self, batchsize=25, max_ingredients=15):
        count = 0
        batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)
        cursor = self.coll.find({}, {"ingredients.foodId": 1, "_id": 0}, no_cursor_timeout = True)

        recipes = np.array(list(cursor))
        np.random.shuffle(recipes)

        for doc in recipes:
            _ids = [int(x['foodId']) for x in doc['ingredients']]
            if len(_ids) > max_ingredients:
                _ids = _ids[:max_ingredients]
            batch[count][:len(_ids)] = _ids[:]
            count += 1

            if count == batchsize:
                yield batch
                count = 0
                batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)

##### Helper functions

In [3]:
def feed_forward_layer(x, target_size, activation_function = None):
    """
    Courtesy of Lukas Braun. Creates a feed forward layer from input x onto size target size.
    """
    print("Forward-Layer:" + str(x.shape))
    
    fan_in = int(x.shape[-1])
    
    if activation_function == tf.nn.relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    weights = tf.get_variable("weights", [x.shape[1], target_size], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [target_size], tf.float32, var_init)
    
    activation = tf.matmul(x, weights) + biases
    
    return activation_function(activation) if callable(activation_function) else activation

def get_ings(embeddings, ingredients):
    '''
    This is used to find the ingredient id to the generated embedding of the lstm.
    We use the cosine similarity to find the closest embedding.
    
    Parameters:
    -----------
    embeddings: 2D Tensor
        the weight matrix with all ingredients representations shape: [ingredient_size, embedding_size]
    ingredients: 3D Tensor
        a matrix of ingedient vectors of who we want to find the nearest neigbhours
        
    Returns:
    --------
    closest_ings: 2D Tensor
        shape [batch_size, max_ingredients]. For each recipe in the batch, it now holds the ingredient ids
      
    '''''
    # we need to normalise for cosine similarity to work
    normed_embedding = tf.nn.l2_normalize(embeddings, dim=1)   

    # unstacking and restacking since matrix multiplications only work on the same rank
    closest_ings = []
    for batch in tf.unstack(ingredients, axis=1):
        batch = tf.nn.l2_normalize(batch, dim=1)
        closest_ings.append(tf.argmax(tf.matmul(batch, tf.transpose(normed_embedding, [1, 0]))))

    closest_ings = tf.stack(closest_ings, axis=1)
    
    return closest_ings

def discriminator_convolution(x, window_size) : 
    """
    Convolutes over the recipe in its embedding form, to extract features.
    Which are used to discriminate between the original recipes and the generated ones.
    
    Params:
    -------
    x: 3D Tensor
        The tensor holding the original recipes and the generated ones with their embeddings
    window_size:  int
        the window size to use in the convolution
        
    Returns:
    --------
    convolution: 2d tensor
        the result of max pooling after the convoliution for feature extraction
    """
    
    x = tf.reshape(x,[2*batch_size, embedding_size, max_ingredients, 1])
    fan_in = 1
    for dim in x.shape[1:]: fan_in * dim.value
    
    var_init = tf.random_normal_initializer(stddev=fan_in**1/2)
    kernel = tf.get_variable("kernel", [embedding_size, window_size, 1,1], tf.float32, var_init)
    
    y = tf.nn.conv2d(x, kernel, strides = [1, 1, 1, 1], padding = "VALID")    
    new_length = y.get_shape()[2]
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [1,1, new_length, 1], initializer = var_init)
    
    y = y + biases
    
    return tf.squeeze(tf.nn.max_pool(y, [1,1, new_length, 1], strides = [1, 1, 1, 1], padding = "VALID"))

def generator_similarity_loss(batch):
    """
    This loss is introduced to make the network choose more diverse ingredients, instead of repeating
    ingredients and using close ingredients
    
    Params:
    -------
    batch: 3d Tensor
        The generated recipe batch
        
    Returns:
    --------
        The mean cosine similarity for each recipe to itself
    """
    def iterate(recipe):
        tn = tf.nn.l2_normalize(recipe, dim=1)
        similarity = tf.matmul(tn, tf.transpose(tn, [1,0]))
        sim = tf.reduce_mean(similarity)
        return sim
    
    all_sims = tf.map_fn(iterate, batch)
        
    
    return tf.stack(all_sims)

##### Loading recipe data, defining hyperparameters

In [4]:
config = "config.json"
recipes = Recipes(config)

with open(config,"r",encoding="utf-8") as cfg:
    settings = json.load(cfg)["network"]

hyperparams = settings["hyperparameters"]
directories = settings["directories"]

epochs = 6
gen_lr = 0.005
dis_lr = 0.001
beta1 = 0.5
batch_size = 50
lambda_ing = 0.4 # coefficient to use for the custom loss

# the pretrained weights
embedding_weights = directories["embedding_weights"]

# for tensorboard
weight_dir = directories["weight_dir"]
summary_dir = directories["summary_dir"]
     
## Hyperparameters
embedding_size = hyperparams["embedding_size"]
# size of the random input vector
z_size = hyperparams["z_size"]
max_ingredients = hyperparams["max_ingredients"]
lstm_memory_size = embedding_size
dropout_rate = hyperparams["dropout_rate"]
optimizer = tf.train.AdamOptimizer

# and create the dictionaries to look up the ingredients.
# unit_size & vocab size are unused and were implemented for further extension of the network
ingredient_size, unit_size, vocabulary_size = recipes.create_dictionaries()

print("""embedding size : {},
input size : {},
max ingredients : {},
lstm state size : {},
dropoutrate : {},
ingredient size : {}"""\
      .format(embedding_size, z_size, max_ingredients, lstm_memory_size, dropout_rate, ingredient_size))

embedding size : 64,
input size : 50,
max ingredients : 10,
lstm state size : 64,
dropoutrate : 0.85,
ingredient size : 3601


##### Defining the graph

In [5]:
# Not resetting causes conflicts in jupyter as the previous executed graph is still active in memory
tf.reset_default_graph()


###### The generator network. ######      
with tf.variable_scope("generator"):

    ##### placeholders  #####
    # for input vector, and lstm state
    with tf.variable_scope("inputs"):
        input_vec = tf.placeholder(tf.float32,[batch_size,z_size],name="input_vec")
        
        cell_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size],name="cell_state")
        hidden_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size],name="hidden_state")
    
    
    ##### embedding #####
    # it is not trainable as it was pre trained to a sufficient degree
    with tf.variable_scope("embedding"):
        init = tf.random_uniform_initializer(-1.0,1.0)
        embeddings = tf.get_variable("embed",
                                     [ingredient_size, embedding_size],
                                     initializer=init,
                                     trainable=False
                                    )
  
        
    ##### RNN #####
    # nothing special about it. We repeatedly feed the input vector for each ingredient to be produced
    with tf.variable_scope("rnn"):
        cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_memory_size)
        cell= tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout_rate)
        zero_state = cell.zero_state(batch_size, tf.float32)
             
        init_state = tf.nn.rnn_cell.LSTMStateTuple(c = cell_state, h = hidden_state)
        inputs = [input_vec for i in range(max_ingredients)]
        
        lstm_out, lstm_state = tf.nn.static_rnn(cell, inputs, initial_state=init_state)
        
        # has shape [max_ingredients, batch_size, embedding], but we need batch_size major
        outputs = tf.transpose(lstm_out, [1,0,2])

    # get trainable variables for generator
    train_gen = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)


##### The discriminator network #####
with tf.variable_scope("discriminator"):
    
    ##### placeholder for originals & batch creation #####
    with tf.variable_scope("input"):
        orig_recipes_placeholder = tf.placeholder(tf.int64,
                                                  [batch_size,max_ingredients],
                                                  name="orig_recipes_placeholder"
                                                 )
        orig_recipes = tf.nn.embedding_lookup(embeddings, orig_recipes_placeholder)

        # create input batch for discriminator
        batch = tf.concat([outputs, orig_recipes],axis=0)
    
    ##### CNN #####
    with tf.variable_scope("convolutions"):
        convs = []
        for i in range(2,max_ingredients+1):
            with tf.variable_scope("conv"+str(i)):
                conv = discriminator_convolution(batch, i)
                convs.append(conv)
        
        conv_outputs = tf.stack(convs, axis=1)
     
    ##### Single Node reduction #####
    with tf.variable_scope("readout"):
        logits = feed_forward_layer(conv_outputs,1,None)
                                    
    # get trainable variables for discriminator
    train_dis = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

##### Evaluation and training of the network #####
with tf.variable_scope("evaluation"):
    
    # to look at what glorious ingredient compilations it produced
    readable_outputs = get_ings(embeddings, outputs)
    
    # to compute cross entropy for generator.
    # consists of only "1" because this is the discriminator label for "real"
    gen_labels = tf.ones((batch_size,1))
    
    # only the first half of the output from the discriminator concerns the pictures
    # produced by the generator
    # so only get first half of logits (which equals the batch_size)
    gen_logits = logits[:batch_size]
    
    # for discriminator cross entropy.
    # first half of input are fake images ("0"), second half real ones ("1")
    dis_labels = tf.concat((tf.zeros((batch_size,1)),tf.ones((batch_size,1))),axis=0)
    dis_logits = logits

    # calculating the loss for generator and discriminator
    gen_loss_mult_all = generator_similarity_loss(outputs)
    gen_loss_multing = tf.reduce_mean(gen_loss_mult_all) * 1#lambda_ing
    gen_loss_ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=gen_labels, logits=gen_logits)
    gen_loss = gen_loss_ce + gen_loss_multing
    
    dis_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=dis_labels, logits=dis_logits)
    
    # initialize optimizer
    gen_optimizer = tf.train.AdamOptimizer(learning_rate=gen_lr, beta1=beta1)
    dis_optimizer = tf.train.AdamOptimizer(learning_rate=dis_lr, beta1=beta1)
    
    # define training steps with respective variable lists
    global_step = tf.get_variable("global_step",[],tf.int32,trainable=False)
    gen_step = gen_optimizer.minimize(gen_loss, var_list=train_gen, global_step=global_step)
    dis_step = dis_optimizer.minimize(dis_loss, var_list=train_dis, global_step=global_step)
    
    # tensorboard things
    tf.summary.scalar("generator_loss", tf.reduce_mean(gen_loss))
    tf.summary.scalar("discriminator_loss", tf.reduce_mean(dis_loss))
    summaries = tf.summary.merge_all()

Forward-Layer:(100, 9)
Instructions for updating:
dim is deprecated, use axis instead


##### This is where we train our network

In [7]:
with tf.Session() as session:
    
    ## initialising directories if they do not exist
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    if not os.path.exists(summary_dir):
        os.makedirs(summary_dir)
        
    # To save what we do. Like a diary, but for ann's
    train_saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(summary_dir, session.graph)
    
    ## Try to load trained data and initialise variables if there is none
    ckpt = tf.train.latest_checkpoint(weight_dir)
    if ckpt is None:
        print("No checkpoint found. Initialising variables.")
        session.run(tf.global_variables_initializer())
    else:
        train_saver.restore(session, ckpt)    
    
    # load the pre trained embedding. If it does not exist, this is not gonna work.
    embed_saver = tf.train.Saver(var_list={'ingredient_embedding': embeddings})
    if os.path.isdir(embedding_weights):
        embed_ckpt = tf.train.latest_checkpoint(embedding_weights)
        if embed_ckpt is not None:
            embed_saver.restore(session, embed_ckpt)
        else:
            raise Exception("Embedding weights not found")
    else:
        raise Exception("Embedding weights folder not found")
    
    
    ## Train all the epochs and save the summaries for each timestep
    total_time = 0
    for epoch in range(1, epochs+1):
        
        print("Starting epoch %d..." %epoch)
        
        # get batch of original recipes #mnist.get_batch(batch_size)
        real_data = recipes.get_ingredient_batch(batch_size, max_ingredients)
        t = time.time()
        
        counter = 0
        
        for data in real_data:
            # lstm initial state will be 0
            _state = session.run(zero_state)
            
            ## for each batch we need a new set of random vectors
            z_vec = np.random.uniform(-1, 1, (batch_size,z_size))            
            
            # feed data into placeholders
            _feed_dict = {input_vec: z_vec, 
                         orig_recipes_placeholder: data,
                         cell_state: _state.c,
                         hidden_state: _state.h,}
            
            # let's train our network!
            _dStep, _gStep, _summ, _step, _fr = session.run([
                                                    dis_step, gen_step,
                                                    summaries, global_step,
                                                    readable_outputs
                                                ],
                                                feed_dict=_feed_dict)
            
            train_writer.add_summary(_summ, _step)
            
            # create the ingredient names. numbers are hard to read
            _fr = recipes.ids2ings(_fr)
            # output every 100 steps            
            if counter % 100 == 0 :
                for rec in np.random.choice(range(batch_size),5):
                    print("|| ",end='')
                    for i in _fr[rec]:
                        print(i," + ",end='')
                    print(" ||")
                print("------------------------------------------")
            
            counter = counter + 1
            
        t = time.time() - t
        total_time += t
        est = t * (epochs - epoch)
        
        ## This is just the time formatting. Move along.
        m, s = divmod(t, 60)
        e_m, e_s = divmod(est, 60)
        
        print(("Finished epoch in {0: .0f}m{1: .2f}s. "+\
              "Estimated remaining time: {2: .0f}m{3: .2f}s").format(m,s,e_m,e_s))
        
        # and save the weights
        train_saver.save(session, weight_dir, global_step=_step)
    
    m, s = divmod(total_time, 60)
    print("Took a total of {0: .0f}m{1: .2f}s".format(m,s))

INFO:tensorflow:Restoring parameters from train_data/weights/-76368
INFO:tensorflow:Restoring parameters from train_data/embedding_weights/model.ckpt
Starting epoch 1...
|| limettensaft  + ingwerpulver  + lauchzwiebel(n)  + zwiebel(n)  + butter  + ananas  + wodka  + blue curaçao  + löwensenf (medium)  + NO_INGREDIENT  +  ||
|| hackfleisch  + ingwerpulver  + NO_INGREDIENT  + paniermehl  + butter  + ananas  + guaranapulver  + tomatensaft  + butter  + grapefruitsaft  +  ||
|| blue curaçao  + ei(er)  + blue curaçao  + zwiebel(n)  + karambolenscheibe(n)  + karambolenscheibe(n)  + salz  + blue curaçao  + butter  + frühlingszwiebel(n)  +  ||
|| blue curaçao  + ei(er)  + blue curaçao  + zwiebel(n)  + karambolenscheibe(n)  + karambolenscheibe(n)  + salz  + blue curaçao  + butter  + frühlingszwiebel(n)  +  ||
|| hackfleisch  + ingwerpulver  + butter  + ananassaft  + gin  + ananas  + knoblauch  + hackfleisch  + pitahaya(s)  + koriandergrün  +  ||
------------------------------------------
|| oliv

|| butter  + zucker, braun  + limette(n)  + blue curaçao  + schafskäse  + galbani parmesan  + limettensaft  + oliven  + zitronensaft  + koriandergrün  +  ||
|| gin  + schafskäse  + karambolenscheibe(n)  + pitu (aguardente de cana)  + paniermehl  + zucker, braun  + limette(n)  + pitahaya(s)  + ananas  + tomate(n)  +  ||
|| champignons  + ananassaft  + koriandergrün  + galbani mozzarella  + honig  + blue curaçao  + honig  + koriandergrün  + löwensenf (medium)  + cognac  +  ||
|| gin  + schafskäse  + karambolenscheibe(n)  + pitu (aguardente de cana)  + paniermehl  + zucker, braun  + limette(n)  + pitahaya(s)  + ananas  + tomate(n)  +  ||
|| ei(er)  + schafskäse  + oliven  + chilischote(n)  + löwensenf (medium)  + grenadine  + wodka  + limette(n)  + wodka  + koriandergrün  +  ||
------------------------------------------
|| lollo rosso  + gin  + NO_INGREDIENT  + zwiebel(n)  + tomatensaft  + chilischote(n)  + frühlingszwiebel(n)  + salatgurke(n)  + chilischote(n)  + möhre(n)  +  ||
|| salz 

|| zucker, braun  + friséesalat  + butter  + pitahaya(s)  + milch  + ei(er)  + knoblauch  + paprikapulver  + ananassaft  + cognac  +  ||
|| paniermehl  + lollo rosso  + champignons  + cognac  + tomatensaft  + NO_INGREDIENT  + wodka  + salatgurke(n)  + chilischote(n)  + paniermehl  +  ||
|| ingwerpulver  + möhre(n)  + NO_INGREDIENT  + löwensenf (medium)  + koriandergrün  + butter  + gemüsebrühe  + champignons  + möhre(n)  + karambolenscheibe(n)  +  ||
|| NO_INGREDIENT  + möhre(n)  + NO_INGREDIENT  + löwensenf (medium)  + paprikapulver  + butter  + gemüsebrühe  + basilikum  + friséesalat  + karambolenscheibe(n)  +  ||
|| grenadine  + aubergine(n)  + cognac  + zucker, braun  + salz  + guaranapulver  + koriandergrün  + zwiebel(n)  + guaranapulver  + tomate(n)  +  ||
------------------------------------------
|| limette(n)  + paniermehl  + galbani mozzarella  + zucker, braun  + NO_INGREDIENT  + limettensaft  + salz  + ei(er)  + pitu (aguardente de cana)  + löwensenf (medium)  +  ||
|| möhre

|| zwiebel(n)  + salatgurke(n)  + limette(n)  + basilikum  + tomate(n)  + lollo rosso  + gin  + zucker, braun  + guaranapulver  + basilikum  +  ||
|| tomate(n)  + salz  + karambolenscheibe(n)  + galbani parmesan  + ei(er)  + ingwerpulver  + galbani parmesan  + aubergine(n)  + blue curaçao  + gin  +  ||
|| zwiebel(n)  + salatgurke(n)  + pitu (aguardente de cana)  + galbani parmesan  + pitu (aguardente de cana)  + salatgurke(n)  + galbani parmesan  + galbani parmesan  + karambolenscheibe(n)  + pitahaya(s)  +  ||
|| appel delikatess mayonnaise  + pitu (aguardente de cana)  + pitu (aguardente de cana)  + ei(er)  + galbani parmesan  + schafskäse  + galbani parmesan  + galbani parmesan  + pitahaya(s)  + wodka  +  ||
|| zwiebel(n)  + salatgurke(n)  + pitu (aguardente de cana)  + basilikum  + paniermehl  + pfeffer  + ananas  + schafskäse  + grenadine  + tomate(n)  +  ||
------------------------------------------
|| karambolenscheibe(n)  + ei(er)  + ei(er)  + möhre(n)  + NO_INGREDIENT  + appel 

|| zitronensaft  + blue curaçao  + hackfleisch  + limettensaft  + limettensaft  + grapefruitsaft  + limette(n)  + limette(n)  + butter  + galbani parmesan  +  ||
|| grapefruitsaft  + schafskäse  + frühlingszwiebel(n)  + pfeffer  + pitahaya(s)  + paniermehl  + koriandergrün  + gemüsebrühe  + butter  + knoblauch  +  ||
|| grapefruitsaft  + tomatensaft  + ananas  + friséesalat  + limettensaft  + grapefruitsaft  + cognac  + limette(n)  + champignons  + grenadine  +  ||
|| paniermehl  + möhre(n)  + guaranapulver  + oliven  + limettensaft  + grapefruitsaft  + limette(n)  + limette(n)  + champignons  + möhre(n)  +  ||
|| gemüsebrühe  + mehl  + salatgurke(n)  + pfeffer  + limettensaft  + salatgurke(n)  + limette(n)  + limette(n)  + champignons  + karambolenscheibe(n)  +  ||
------------------------------------------
|| aubergine(n)  + wodka  + ananas  + hackfleisch  + ingwerpulver  + salz  + grenadine  + salatgurke(n)  + hackfleisch  + galbani mozzarella  +  ||
|| gemüsebrühe  + salz  + ananas

...

|| zucker, braun  + lauchzwiebel(n)  + hackfleisch  + lollo rosso  + hackfleisch  + guaranapulver  + galbani mozzarella  + champignons  + pitu (aguardente de cana)  + pitahaya(s)  +  ||
|| knoblauch  + milch  + wodka  + zitronensaft  + pitahaya(s)  + pfeffer  + pfeffer  + salz  + milch  + zitronensaft  +  ||
|| pfeffer  + guaranapulver  + hackfleisch  + ananas  + lollo rosso  + schafskäse  + pfeffer  + paprikapulver  + honig  + zitronensaft  +  ||
|| ananas  + hackfleisch  + tomate(n)  + löwensenf (medium)  + grapefruitsaft  + gemüsebrühe  + NO_INGREDIENT  + schafskäse  + paprikapulver  + mehl  +  ||
|| pfeffer  + guaranapulver  + hackfleisch  + ananas  + lauchzwiebel(n)  + schafskäse  + pfeffer  + paprikapulver  + ingwerpulver  + zitronensaft  +  ||
------------------------------------------
|| lauchzwiebel(n)  + aubergine(n)  + gin  + lauchzwiebel(n)  + lauchzwiebel(n)  + karambolenscheibe(n)  + karambolenscheibe(n)  + appel delikatess mayonnaise  + cognac  + ingwerpulver  +  ||
|| a

|| ei(er)  + limette(n)  + ananas  + limette(n)  + ingwerpulver  + ananas  + karambolenscheibe(n)  + aubergine(n)  + limette(n)  + honig  +  ||
|| mehl  + zwiebel(n)  + limette(n)  + limette(n)  + ingwerpulver  + ananas  + friséesalat  + grapefruitsaft  + limette(n)  + honig  +  ||
|| butter  + honig  + wodka  + gin  + basilikum  + NO_INGREDIENT  + basilikum  + butter  + galbani parmesan  + galbani mozzarella  +  ||
|| karambolenscheibe(n)  + zwiebel(n)  + tomatensaft  + limette(n)  + frühlingszwiebel(n)  + schafskäse  + ei(er)  + ei(er)  + gin  + galbani parmesan  +  ||
|| lollo rosso  + NO_INGREDIENT  + wodka  + zucker, braun  + salz  + paniermehl  + galbani mozzarella  + salz  + galbani parmesan  + NO_INGREDIENT  +  ||
------------------------------------------
|| salz  + aubergine(n)  + zwiebel(n)  + zwiebel(n)  + zwiebel(n)  + wodka  + ananas  + frühlingszwiebel(n)  + galbani parmesan  + NO_INGREDIENT  +  ||
|| gemüsebrühe  + frühlingszwiebel(n)  + ananassaft  + hackfleisch  + sal

|| tomatensaft  + tomate(n)  + frühlingszwiebel(n)  + knoblauch  + paprikapulver  + butter  + pitahaya(s)  + salz  + pitahaya(s)  + salz  +  ||
|| löwensenf (medium)  + galbani parmesan  + frühlingszwiebel(n)  + lauchzwiebel(n)  + lauchzwiebel(n)  + zitronensaft  + pitahaya(s)  + pitahaya(s)  + grenadine  + NO_INGREDIENT  +  ||
|| löwensenf (medium)  + cognac  + frühlingszwiebel(n)  + lauchzwiebel(n)  + champignons  + pitahaya(s)  + pitahaya(s)  + salz  + löwensenf (medium)  + blue curaçao  +  ||
|| löwensenf (medium)  + cognac  + frühlingszwiebel(n)  + lauchzwiebel(n)  + champignons  + limette(n)  + blue curaçao  + schafskäse  + löwensenf (medium)  + pfeffer  +  ||
|| löwensenf (medium)  + cognac  + frühlingszwiebel(n)  + lauchzwiebel(n)  + champignons  + pitahaya(s)  + pitahaya(s)  + salz  + grenadine  + blue curaçao  +  ||
------------------------------------------
|| zitronensaft  + wodka  + wodka  + salatgurke(n)  + paprikapulver  + ananassaft  + ei(er)  + löwensenf (medium)  + lo

|| karambolenscheibe(n)  + ananas  + ingwerpulver  + galbani parmesan  + oliven  + knoblauch  + ei(er)  + oliven  + aubergine(n)  + ananas  +  ||
|| karambolenscheibe(n)  + grenadine  + paprikapulver  + aubergine(n)  + paprikapulver  + ananassaft  + oliven  + ananas  + knoblauch  + grapefruitsaft  +  ||
|| karambolenscheibe(n)  + ananas  + knoblauch  + ananas  + oliven  + guaranapulver  + ei(er)  + ingwerpulver  + cognac  + galbani parmesan  +  ||
|| karambolenscheibe(n)  + grenadine  + ingwerpulver  + pfeffer  + ananassaft  + ananassaft  + oliven  + gin  + lauchzwiebel(n)  + ananassaft  +  ||
|| karambolenscheibe(n)  + grenadine  + löwensenf (medium)  + knoblauch  + paprikapulver  + oliven  + knoblauch  + chilischote(n)  + chilischote(n)  + basilikum  +  ||
------------------------------------------
|| basilikum  + appel delikatess mayonnaise  + paprikapulver  + zitronensaft  + champignons  + cognac  + cognac  + gin  + pfeffer  + zwiebel(n)  +  ||
|| wodka  + appel delikatess mayonnai

|| löwensenf (medium)  + frühlingszwiebel(n)  + lauchzwiebel(n)  + ananassaft  + honig  + aubergine(n)  + tomatensaft  + lauchzwiebel(n)  + mehl  + paniermehl  +  ||
|| löwensenf (medium)  + frühlingszwiebel(n)  + lauchzwiebel(n)  + ananassaft  + honig  + aubergine(n)  + tomatensaft  + lauchzwiebel(n)  + mehl  + paniermehl  +  ||
|| löwensenf (medium)  + tomate(n)  + salz  + ananassaft  + honig  + ingwerpulver  + tomatensaft  + aubergine(n)  + ei(er)  + paniermehl  +  ||
|| löwensenf (medium)  + pfeffer  + lauchzwiebel(n)  + ananassaft  + lollo rosso  + aubergine(n)  + aubergine(n)  + lauchzwiebel(n)  + mehl  + hackfleisch  +  ||
|| basilikum  + appel delikatess mayonnaise  + grapefruitsaft  + zucker, braun  + gemüsebrühe  + löwensenf (medium)  + grenadine  + limettensaft  + grapefruitsaft  + gemüsebrühe  +  ||
------------------------------------------
|| limette(n)  + chilischote(n)  + ananassaft  + limette(n)  + limette(n)  + grapefruitsaft  + grapefruitsaft  + zwiebel(n)  + paprika